# ROS2 Documentation for Beginners
## Chapter 2 Core of ROS2 Communication Mechanism

A robot is a highly complex, systematic implementation. A complete robotic application may consist of several functional modules, and each module may contain multiple functions. To operate effectively, these different modules and functions need to exchange data frequently. For instance, in the navigation system's path planning module:

During path planning, data from other modules is needed as input, and the output is also provided for use by other modules.

The input data includes map data provided by a mapping service, robot pose data provided by the localization module, target point data provided by the human-machine interaction module, and so on.

The output path information can then be subscribed to by the motion control module or displayed on the human-machine interaction interface.

So, how do these relatively independent functional modules or points exchange data with one another? To answer this, we need to introduce the communication mechanism in ROS 2.


### Overview of this chapter ### 

| Chapter | Learning Content                  | Learning Outcomes |
| ------- | --------------------------------- | ----------------- |
| 2.1     | Introduction to Communication Mechanism | Terminology involved in the communication mechanism. | Understand common ROS 2 communication terms, such as nodes, topics, interfaces, and the data exchange flow in different communication models. |
| 2.2     | Topic Communication               | Concept, function, application scenarios, and code implementation of topic communication. | Understand the application scenarios of topic communication, related concepts (publisher, subscriber, message interface, topic...), and master the implementation process, including writing C++ and Python code. |
| 2.3     | Service Communication             | Concept, function, application scenarios, and code implementation of service communication. | Understand the application scenarios of service communication, related concepts (server, client, service interface, topic...), and master the implementation process, including writing C++ and Python code. |
| 2.4     | Action Communication              | Concept, function, application scenarios, and code implementation of action communication. | Understand the application scenarios of action communication, related concepts (action server, action client, action interface, topic...), and master the implementation process, including writing C++ and Python code. |
| 2.5     | Parameter Service                 | Concept, function, application scenarios, and code implementation of parameter service. | Understand the application scenarios of parameter service, and master the implementation process, including writing C++ and Python code. |
| 2.6     | Chapter Summary                   | Summary of key points. | Review of key points. |


## 2.1 ROS2 Communication Mechanism Introduction


In ROS 2, although there are various communication methods, the components of each method are similar. For example: communication involves two or more parties, different communication objects need to be linked, each has its own model, and data exchange always requires a data carrier. This section will introduce some of the terminology involved in ROS 2 communication.

### 2.1.1 Node

During the communication, the basic unit of ROS 2 is the node. A node is a process that performs a specific function. For example, a node can be a sensor driver, a controller, or a planner. Nodes communicate with each other by sending and receiving messages. A ROS 2 system is composed of multiple nodes.

### 2.1.2 Topic

A topic is a named bus over which nodes exchange messages. A node can publish messages to a topic, subscribe to a topic to receive messages, or do both. Topics are used for one-to-many communication. For example, a sensor node publishes messages to a topic, and multiple nodes subscribe to the topic to receive the messages.
### 2.1.3 Communication Models

After different communication entities are linked through a topic, how is the communication actually implemented? In ROS 2, there are four commonly used communication models:

**1. Topic Communication:** A one-way communication model where, between the two parties, the publisher sends data and the subscriber receives data. The data flow is one-directional, from the publisher to the subscriber.

**2. Service Communication:** A request-response communication model. Here, the client sends request data to the server, and the server responds with the result back to the client.

**3. Action Communication:** A communication model with continuous feedback. In this model, the client sends a request to the server, and the server responds with the final result. However, during the time between receiving the request and producing the final response, the server continuously sends feedback updates to the client.

**4. Parameter Service:** A shared-state communication model. Here, the server can set data, and the client can connect to the server to read or modify the server’s data.

### 2.1.4 Interface

In ROS 2, the data exchanged between nodes is defined by an interface. An interface is a description of the data structure, including the message type, message fields, and message data. The interface is defined in a file with the extension `.msg` for messages, `.srv` for services, and `.action` for actions.

```plaintext
# '.msg' 
int64 num1
int64 num2
```

```plaintext
# '.srv'
int64 num1
int64 num2
---
int64 sum
```

```plaintext
# '.action'
int64 num1
---
int64 num2
---
float64 progress
``` 
#### Data strucutre of the message, service and action interface
It can be different types, like int, float, string, etc. The data structure of the message, service, and action interface is defined in the `.msg`, `.srv`, and `.action` files, respectively.

### 2.1.5 Prepare for the exercise
1. mkdir a new workspace named "ws01_plumbing"
2. in ```bash src``` folder, create a new package named base_interfaces_demo by using the command ```ros2 pkg create --build-type ament_cmake base_interfaces_demo```

## 2.2 Topic Communication
**Scenario**

Topic communication is the most frequently used communication model in ROS and is based on a publish-subscribe model, where one node publishes messages, and another node subscribes to these messages. Topic communication is widely applicable in various scenarios, such as the following:

*When a robot performs navigation using a LiDAR sensor, it collects data from the sensor, processes it, and generates motion control commands to drive the robot’s chassis.*

In this case, topic communication is used multiple times:

**LiDAR Data Collection and Processing:** In ROS, a node continuously publishes the latest data collected by the LiDAR sensor. Meanwhile, a node within the navigation module subscribes to and interprets this data.

**Publishing Motion Commands:** The navigation module calculates motion control commands by integrating multiple data sources and publishes them to the chassis control module. The chassis control node subscribes to the motion commands and converts them into pulse signals to control the motors.

Likewise, data from sensors like LiDAR, cameras, GPS, etc., is collected using topic communication. Topic communication is well-suited for applications requiring continuous data updates.

**Concepts**

Topic communication is based on the publish-subscribe model. In this model, a node publishes messages to a topic, and another node subscribes to the topic to receive the messages. The publisher and subscriber nodes are decoupled, meaning they do not need to know each other’s existence. The publisher node only needs to know the topic name, and the subscriber node only needs to know the topic name and the message type.

<div style="text-align: center;">
    <img src="figs/2_2Topic_Communicaiton_model_1.gif" alt="Topic Communication Model" >
</div>

The above example is one-to-one relation. In ROS2, topic communication can also be one-to-many. In this case, multiple nodes can subscribe to the same topic to receive messages from the publisher node.

<div style="text-align: center;">
    <img src="figs/2_2Topic_Communicaiton_model_2.gif" alt="Topic Communication Model" >
</div>


**Purpose**

Topic communication is typically used for scenarios involving frequently updated data that require minimal logic processing.

**About Message Interfaces**

There are several ways to work with message interfaces:

- **Using Standard Data Types:** In ROS 2, the std_msgs package includes some basic native data types, such as String, Int8, Int16, Int32, Int64, Float32, Float64, Char, Bool, and Empty. These native types can serve as carriers for topic communication, though they generally contain only a single data field. Since the interface files in std_msgs are relatively simple, they have limited functionality. When transmitting complex data structures, they may prove inadequate.

- **Standard Topic Message Interfaces:** ROS 2 also provides many pre-defined standard topic message interfaces, which are widely used in practical applications. For example, the sensor_msgs package includes various sensor-related message interfaces (for LiDAR, cameras, point clouds, etc.), while the geometry_msgs package defines interfaces for geometric messages (points, coordinate frames, velocity commands, etc.).

- **Custom Message Interfaces:** If the above interface files don’t meet specific needs, you can create custom message interfaces to fit those requirements.


### 2.2.1 Exmaple of Topic Communication with Default Message Interface

<img src="Examples/excution_demo.gif" alt="Execution Demo" width="300" autoplay>


### Explaination of the Example

In this example, we will create two nodes: a publisher node and a subscriber node. The publisher node will publish messages to the topic, and the subscriber node will subscribe to the topic to receive the messages. The publisher node will publish messages containing the current time, and the subscriber node will display the received messages.


### Create a default message interface ###
Under ```bash src``` folder, create a new package with the following command:
```bash
ros2 pkg create py01_topic --build-type ament_python --dependencies rclpy std_msgs base_interfaces_demo
```

**talker_node**
```python
"""
    Requirements: Send the text "hello world!" at a fixed frequency,
    appending an incrementing number to the end of each message.

    Steps:

    1. Import necessary packages
    2. Initialize the ROS 2 client
    3. Define the node class
        3-1. Create the publisher
        3-2. Create a timer
        3-3. Construct and publish the message
    4. Call the spin function and pass in the node object
    5. Release resources
"""

# import necessary packages
import rclpy
from rclpy.node import Node
from std_msgs.msg import String


# Define the node class
class MinimalPublisher(Node):

    def __init__(self):
        super().__init__('minimal_publisher_py')
        # create the publisher
        self.publisher_ = self.create_publisher(String, 'topic', 10)
        # create a timer
        timer_period = 0.5
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.i = 0
    # construct and publish the message
    def timer_callback(self):
        msg = String()
        msg.data = 'hello world! %d' % self.i
        self.publisher_.publish(msg)
        self.get_logger().info('Publishing: "%s"' % msg.data)
        self.i += 1
        
        
def main(args=None):
    # initialize the ROS 2 client
    rclpy.init(args=args)
    minimal_publisher = MinimalPublisher()
    rclpy.spin(minimal_publisher)
    rclpy.shutdown()
    # release resources
    minimal_publisher.destroy_node()
    
if __name__ == '__main__':
    main()    
```

**listener_node**
```python
"""
    Requirements: Receive messages from the topic and display them.

    Steps:

    1. Import necessary packages
    2. Initialize the ROS 2 client
    3. Define the node class
        3-1. Create the subscriber
        3-2. Define the callback function
    4. Call the spin function and pass in the node object
    5. Release resources
"""
"""
Requirements: Subscribe to the messages published by the publisher and print them to the terminal.

Steps:

    1. Import necessary packages
    2. Initialize the ROS 2 client
    3. Define the node class
        3-1. Create the subscriber
        3-2. Process the received messages
    4. Call the spin function and pass in the node object
    5. Release resources
"""


# import necessary packages
import rclpy
from rclpy.node import Node
from std_msgs.msg import String


# Define the node class
class MinimalSubscriber(Node):

    def __init__(self):
        super().__init__('minimal_subscriber_py')
        # create the subscriber
        self.subscription = self.create_subscription(
            String,
            'topic',
            self.listener_callback,
            10)
        self.subscription  # prevent unused variable warning

    # process the received messages
    def listener_callback(self, msg):
        self.get_logger().info('I heard: "%s"' % msg.data)
        

def main(args=None):
    # initialize the ROS 2 client
    rclpy.init(args=args)
    minimal_subscriber = MinimalSubscriber()
    rclpy.spin(minimal_subscriber)
    rclpy.shutdown()
    # release resources
    minimal_subscriber.destroy_node()

if __name__ == '__main__':
    main()
```

**Edit configuration file**
**package.xml**
```xml
  <depend>rclpy</depend>
  <depend>std_msgs</depend>
  <depend>base_interfaces_demo</depend>
```

**Edit the setup.py file**
```python

    entry_points={
        'console_scripts': [
            'demo01_talker_str_py = py01_topic.demo01_talker_str_py:main',
            'demo02_listener_str_py = py01_topic.demo02_listener_str_py:main'
        ],
    },
```


** Execution **


**Terminal 1**
```bash
. install/setup.bash
ros2 run base_interfaces_demo demo01_talker_str_py
```

**Terminal 2**
```bash
. install/setup.bash
ros2 run base_interfaces_demo demo02_listener_str_py
```

###  Workflow of the Default Example
1. Create the publisher node
2. Create the subscriber node
3. Edit the setup.py file
4. Complie
5. Run


### 2.2.2 Example of Topic Communication with Custom Message Interface

<img src="Examples/2_2_2_2_Customer_Interface_Topic_example.gif" alt="Execution Demo" width="320" autoplay>

### Explaination of the Example ###

### Create a custom message interface ###
1. Create and edit ```bash .msg``` file
2. Edit the ```bash package.xml``` file 
3. Edit the ```CMakeLists.txt``` file
4. Test the custom message interface

1. Create and edit the .msg file
```plaintext
string   name
int32    age
float64  height
```
2. Edit the CMakelists.txt file
```plaintext
rosidl_generate_interfaces(${PROJECT_NAME}
"msg/Student.msg"
)
```

3. Compile
```bash
colcone build --packages-select base_interfaces_demo
```

4. Test the custom message interface
   
``` bash
. install/setup.bash
ros2 interface show base_interfaces_demo/msg/Student
```

### Define the publisher ###

```python
"""
requirement:
send student's information including
name, age, and score based on the frequencey.
"""


# import required libraries
import rclpy
from rclpy.node import Node
from base_interfaces_demo.msg import Student


# define the node
class MinimalPublisher(Node):
    
    
    def __init__(self):
        super().__init__('stu_publisher_py')
        self.publisher_ = self.create_publisher(Student, 'topic_stu', 10)
        # define timer
        timer_period = 0.25
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.i = 0
        
        
    def timer_callback(self):
        stu = Student()
        stu.name = 'student' + 'your_wsu_id'
        stu.age = self.i
        stu.score = 'A'
        self.publisher_.publish(stu)
        self.get_logger().info('Publishing: name=%s,age= %d, score=%s' % (stu.name, stu.age, stu.score))
        self.i += 1
        
def main(args=None):
    # initialize the node
    rclpy.init(args=args)
    minimal_publisher = MinimalPublisher()
    # using the spin function
    rclpy.spin(minimal_publisher)
    minimal_publisher.destroy_node()
    rclpy.shutdown()
    
    
if __name__ == '__main__':
    main()
    
    
```

### Define the subscriber ###

```python
"""
requirements:subscribe the student's information and print it out.
"""
# import required libraries
import rclpy
from rclpy.node import Node
from base_interfaces_demo.msg import Student


# define the node
class MinimalSubscriber(Node):
    
    
    def __init__(self):
        super().__init__('stu_subscriber_py')
        # create the subscription
        self.subscription = self.create_subscription(Student,
                                                     'topic_stu',
                                                     self.listener_callback,
                                                     10)
        self.subscription  # prevent unused variable warning
        
        
        
    def listener_callback(self, msg):
        self.get_logger().info('I heard: name=%s, age=%d, score=%s' % (msg.name, msg.age, msg.score))
        

def main(args=None):
    # initialize the node
    rclpy.init(args=args)
    minimal_subscriber = MinimalSubscriber()
    # using the spin function
    rclpy.spin(minimal_subscriber)
    minimal_subscriber.destroy_node()
    rclpy.shutdown()
    
if __name__ == '__main__':
    main()
    

```

### Edit the configuration file

The **package.xml** needs no changes. However, we need to edit the **setup.py** file.

```python
    entry_points={
        'console_scripts': [
            'demo03_talker_stu_py = py01_topic.demo03_talker_stu_py:main',
            'demo04_listener_stu_py = py01_topic.demo04_listener_stu_py:main'
        ],
    },
```

To new **setup.py** file will look like this:

```python

entry_points={
    'console_scripts': [
        'demo01_talker_str_py = py01_topic.demo01_talker_str_py:main',
        'demo02_listener_str_py = py01_topic.demo02_listener_str_py:main',
        'demo03_talker_stu_py = py01_topic.demo03_talker_stu_py:main',
        'demo04_listener_stu_py = py01_topic.demo04_listener_stu_py:main'
    ],
},
```

### Compile ###
```bash
colcon build --packages-select py01_topic
```

### Execution ###

**Terminal 1**
```bash
. install/setup.bash
ros2 run base_interfaces_demo demo03_talker_stu_py
```
**Terminal 2**
```bash
. install/setup.bash
ros2 run base_interfaces_demo demo04_listener_stu_py
```





## 2.2 Service Communication

**Scenario**

Service communication is another very porpuler communication model in ROS2. It is based on the request-response model, where one node sends a request to another node, and the latter responds with the result. Service communication is widely used in scenarios where a node needs to request data from another node, and the response is required to continue the operation.

For example, in a robot's navigation system, the path planning module may need to request the map data from the mapping module. The path planning module sends a request to the mapping module, and the mapping module responds with the map data. The path planning module then uses this data to plan the path.

Based on the previous example, the path planning module is the client, and the mapping module is the server. The client sends a request to the server, and the server responds with the result. The client and server nodes are decoupled, meaning they do not need to know each other's existence. The client node only needs to know the service name and the service type, while the server node only needs to know the service name and the service type.

**Concepts**

Service communication is based on the request-response model. In this model, a node sends a request to another node, and the latter responds with the result. The client node sends a request to the server node, and the server node processes the request and sends the response back to the client node.

<div style="text-align: center;">
    <img src="figs/2_3Service_Communicaiton_model_1.gif" alt="Service Communication Model" >
</div>

During service communication, the client node sends a request to the server node, and the server node processes the request and sends the response back to the client node. The client node then uses the response to continue its operation.

<div style="text-align: center;">
    <img src="figs/2_3Service_Communicaiton_model_2.gif" alt="Service Communication Model" >
</div>

**Purpose**

Service communication is typically used in scenarios where a node needs to request data from another node, and the response is required to continue the operation.

### Example of Service Communication ###

<div style = "text-align:center;">
    <img src = "figs/chapter_2_service_communication_demo.gif" alt="Service Communication Demo">
</div>


### Explaination of the Example ###

In this example, we will create two nodes: a server node and a client node. The server node will provide a service that returns the sum of two numbers, and the client node will send a request to the server node to calculate the sum.

### Edit the interface file ###

1. Create and edit the .srv file
2. Edit the package.xml file
3. Edit the CMakeLists.txt file
4. Test the custom service interface

1. Create and edit the .srv file
```plaintext
int32 num1
int32 num2
---
int32 sum
```

2. Edit the package.xml file

Since we will use the same package.xml, so wed don't need to modify it.
```xml
  <build_depend>rosidl_default_generators</build_depend>
  <depend>rclpy</depend>
  <depend>std_msgs</depend>
  <depend>base_interfaces_demo</depend>
```

3. Edit the CMakeLists.txt file
```plaintext

rosidl_generate_interfaces(${PROJECT_NAME}
  "msg/Student.msg"
  "srv/AddInts.srv"
)
```

4. Test
```bash

. install/setup.bash
ros2 interface show base_interfaces_demo/srv/AddInts

```

The output will be:

<div style = "text-align:center;">
    <img src = "figs/service_communication_srv_result.png" alt="Service Communication Demo">
</div>


### Workflow of the Example ###

1. Create the server node
2. Create the client node
3. Edit the setup.py file
4. Compile
5. Run

